# Bayesian Classification for ECG Time-Series

> Copyright 2019 Dave Fernandes. All Rights Reserved.
> 
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
>
> http://www.apache.org/licenses/LICENSE-2.0
>  
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

## Overview
This notebook classifies time-series for segmented heartbeats from ECG lead II recordings. Either of two models (CNN or RNN) can be selected from training and evaluation.
- Data for this analysis should be prepared using the `PreprocessECG.ipynb` notebook from this project.
- Original data is from: https://www.kaggle.com/shayanfazeli/heartbeat

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as keras
import matplotlib.pyplot as plt
import pickle

# tf.enable_eager_execution()

from google.colab import drive
drive.mount('/content/drive')

TRAIN_SET = '/content/drive/My Drive/Colab Notebooks/Data/train_set.pickle'
TEST_SET = '/content/drive/My Drive/Colab Notebooks/Data/test_set.pickle'

with open(TEST_SET, 'rb') as file:
    test_set = pickle.load(file)
    x_test = test_set['x']
    y_test = test_set['y']

with open(TRAIN_SET, 'rb') as file:
    train_set = pickle.load(file)
    x_train = train_set['x']
    y_train = train_set['y']
    
print('Train size:', len(y_train), 'Test size:', len(y_test))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train size: 441841 Test size: 500


### Input Datasets

In [0]:
def combined_dataset(features, labels):
    assert features.shape[0] == labels.shape[0]
    dataset = tf.data.Dataset.from_tensor_slices((np.expand_dims(features, axis=-1), labels))
    return dataset

# For training
def train_input_fn():
    dataset = combined_dataset(x_train, y_train)
    return dataset.repeat().shuffle(500000).batch(256, drop_remainder=True).prefetch(1)

# For evaluation and metrics
def eval_input_fn():
    dataset = combined_dataset(x_test, y_test)
    return dataset.repeat().batch(500).prefetch(1)

### Define the model
#### Bayesian CNN Model
* The convolutional model is taken from: https://arxiv.org/pdf/1805.00794.pdf

Model consists of:
* An initial 1-D convolutional layer
* 5 repeated residual blocks (`same` padding)
* A fully-connected layer
* A linear layer with softmax output
* Flipout layers are used instead of standard layers

In [0]:
MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/Models/CNN'

def conv_unit(unit, input_layer):
    s = '_' + str(unit)
    layer = keras.Convolution1D(name='Conv1' + s, filters=32, kernel_size=5, strides=1, padding='same', activation='relu')(input_layer)
    layer = keras.Convolution1D(name='Conv2' + s, filters=32, kernel_size=5, strides=1, padding='same', activation=None)(layer )
    layer = keras.Add(name='ResidualSum' + s)([layer, input_layer])
    layer = keras.Activation("relu", name='Act' + s)(layer)
    layer = keras.MaxPooling1D(name='MaxPool' + s, pool_size=5, strides=2)(layer)
    return layer

def cnn_model(input_shape):
    time_series = keras.Input(shape=input_shape, dtype='float32')
    current_layer = keras.Convolution1D(filters=32, kernel_size=5, strides=1)(time_series)

    for i in range(5):
        current_layer = conv_unit(i + 1, current_layer)

    current_layer = keras.Flatten()(current_layer)
    current_layer = keras.Dense(32, name='FC1', activation='relu')(current_layer)
    logits = keras.Dense(5, name='Output')(current_layer)
    
    model = tf.keras.Model(inputs=time_series, outputs=logits, name='cnn_model')
    return model
  
# Standard cross-entropy loss
categorical_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
def loss_fn(y_true, y_pred):
    log_likelihood_loss = categorical_loss(y_true, y_pred)
    return log_likelihood_loss

In [4]:
import tensorflow_probability as tfp

BAYES_MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/Models/BayesianCNN'

def bayes_conv_unit(unit, input_layer):
    s = '_' + str(unit)
    layer = tfp.layers.Convolution1DFlipout(name='Conv1' + s, filters=32, kernel_size=5, strides=1, padding='same', activation='relu')(input_layer)
    layer = tfp.layers.Convolution1DFlipout(name='Conv2' + s, filters=32, kernel_size=5, strides=1, padding='same', activation=None)(layer )
    layer = keras.Add(name='ResidualSum' + s)([layer, input_layer])
    layer = keras.Activation("relu", name='Act' + s)(layer)
    layer = keras.MaxPooling1D(name='MaxPool' + s, pool_size=5, strides=2)(layer)
    return layer

def bayes_cnn_model(input_shape):
    time_series = tf.keras.layers.Input(shape=input_shape, dtype='float32')
    current_layer = keras.Convolution1D(filters=32, kernel_size=5, strides=1)(time_series)

    for i in range(5):
        current_layer = conv_unit(i + 1, current_layer)

    current_layer = keras.Flatten()(current_layer)
    current_layer = keras.Dense(32, name='FC1', activation='relu')(current_layer)
    logits = tfp.layers.DenseFlipout(5, name='Output')(current_layer)
    
    model = tf.keras.Model(inputs=time_series, outputs=logits, name='bayes_cnn_model')
    return model
  
# Compute the -ELBO as the loss, averaged over the batch size.
bayes_categorical_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
def bayes_loss_fn(y_true, y_pred):
    log_likelihood_loss = bayes_categorical_loss(y_true, y_pred)
    kl = sum(model.losses) / 1725
    return log_likelihood_loss + kl


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



### Train model

In [5]:
import tensorflow.distributions as tfd

# Initial learning rate
INITIAL_LEARNING_RATE = 0.001

# Learning rate decay per LR_DECAY_STEPS steps (1.0 = no decay)
LR_DECAY_RATE = 0.5

# Number of steps for LR to decay by LR_DECAY_RATE
LR_DECAY_STEPS = 4000

model = bayes_cnn_model([187, 1])

# global_step = tf.train.get_global_step()
# learning_rate = tf.train.exponential_decay(INITIAL_LEARNING_RATE, global_step, LR_DECAY_STEPS, LR_DECAY_RATE)
learning_rate = INITIAL_LEARNING_RATE
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

model.compile(loss=bayes_loss_fn, optimizer=optimizer, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_input_fn(), validation_data=eval_input_fn(), epochs=2, steps_per_epoch=1725, validation_steps=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Epoch 1/2
1725/1725 [==============================] - 39s 23ms/step - loss: 276.0610 - sparse_categorical_accuracy: 0.8129 - val_loss: 156.1742 - val_sparse_categorical_accuracy: 0.8000
Epoch 2/2
1725/1725 [==============================] - 36s 21ms/step - loss: 73.2309 - sparse_categorical_accuracy: 0.4604 - val_loss: 17.2184 - val_sparse_categorical_accuracy: 0.1500


### Compute metrics

In [6]:
import sklearn.metrics as skm

x = np.expand_dims(x_test, axis=-1)
y_logits = model.predict(x)
y_predicted = np.argmax(y_logits, axis=1)
y_test = np.reshape(y_test, (len(y_test), 1))

# Classification report
report = skm.classification_report(y_test, y_predicted)
print(report)

# Confusion matrix
conf = skm.confusion_matrix(y_test, y_predicted)
print(conf)

              precision    recall  f1-score   support

           0       0.17      0.01      0.02       100
           1       0.00      0.00      0.00       100
           2       0.25      0.02      0.04       100
           3       0.06      0.03      0.04       100
           4       0.17      0.72      0.27       100

   micro avg       0.16      0.16      0.16       500
   macro avg       0.13      0.16      0.07       500
weighted avg       0.13      0.16      0.07       500

[[ 1  0  1  3 95]
 [ 0  0  0  6 94]
 [ 4  0  2 22 72]
 [ 0  0  0  3 97]
 [ 1  2  5 20 72]]


In [7]:
correct = []
false_pos = []
false_neg = []

for k in range(5):
    for i in range(len(y_test)):
        if y_test[i] == k and y_predicted[i] == k:
            correct.append(y_prob[i, k])
        elif y_test[i] == k and y_predicted[i] != k:
            false_neg.append(y_prob[i, k])
        elif y_test[i] != k and y_predicted[i] == k:
            false_pos.append(y_prob[i, k])

n, bins, patches = plt.hist(correct, 20, (0, 1))
plt.xlabel('Probability')
plt.title('Correctly Classified')
plt.show();

n, bins, patches = plt.hist(false_pos, 20, (0, 1))
plt.xlabel('Probability')
plt.title('False Positives')
plt.show();

n, bins, patches = plt.hist(false_neg, 20, (0, 1))
plt.xlabel('Probability')
plt.title('False Negatives')
plt.show()

NameError: ignored